https://github.com/gusmartinezr/IASIS421/tree/main/Examen/Examen%20Final

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
f = open("/content/drive/MyDrive/IA2/ApodosIngles.txt", "r", encoding='utf-8')
text = f.read()
text[:300], len(text)

('Aaron\tRon RonnE Erin\tRonald\nAbel\tAbe Ab Eb EbbE\nAbiel\tBiel Ab\nAbigail\tAb AbbE Gail NabbE\tTabitha\nAbijah\tAb Bige Abiah Ab Biah\nAbner\tAb AbbE\nAbraham\tAbe Abram\nAbsalom\tAb AbbE\nAdaline\tAda Adela Aline Edith AddE Dell Delia Lena\tAdelaide\nAdam\tAde EdE Ad\nAdelaide\tAddE Adela Adaline Adeline Della Heide Ad',
 16755)

## Tokenización

In [22]:
import string
all_characters = string.printable
all_characters

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [23]:
import string

class Tokenizer(): 
    
  def __init__(self):
    self.all_characters = all_characters
    self.n_characters = len(self.all_characters)
    
  def text_to_seq(self, string):
    seq = []
    for c in range(len(string)):
        try:
            seq.append(self.all_characters.index(string[c]))
        except:
            continue
    return seq

  def seq_to_text(self, seq):
    text = ''
    for c in range(len(seq)):
        text += self.all_characters[seq[c]]
    return text

tokenizer = Tokenizer()
tokenizer.n_characters

100

In [24]:
tokenizer.text_to_seq('Aaron')

[36, 10, 27, 24, 23]

In [25]:
tokenizer.seq_to_text([36, 10, 27, 24, 23])

'Aaron'

In [26]:
text_encoded = tokenizer.text_to_seq(text)

In [27]:
train_size = len(text_encoded) * 80 // 100 
train = text_encoded[:train_size]
test = text_encoded[train_size:]

len(train), len(test)

(13404, 3351)

In [28]:
import random

def windows(text, window_size = 100):
    start_index = 0
    end_index = len(text) - window_size
    text_windows = []
    while start_index < end_index:
      text_windows.append(text[start_index:start_index+window_size+1])
      start_index += 1
    return text_windows

text_encoded_windows = windows(text_encoded)

In [29]:
print(tokenizer.seq_to_text((text_encoded_windows[0])))
print()
print(tokenizer.seq_to_text((text_encoded_windows[1])))
print()
print(tokenizer.seq_to_text((text_encoded_windows[2])))

Aaron	Ron RonnE Erin	Ronald
Abel	Abe Ab Eb EbbE
Abiel	Biel Ab
Abigail	Ab AbbE Gail NabbE	Tabitha
Abij

aron	Ron RonnE Erin	Ronald
Abel	Abe Ab Eb EbbE
Abiel	Biel Ab
Abigail	Ab AbbE Gail NabbE	Tabitha
Abija

ron	Ron RonnE Erin	Ronald
Abel	Abe Ab Eb EbbE
Abiel	Biel Ab
Abigail	Ab AbbE Gail NabbE	Tabitha
Abijah


In [30]:
import torch

class CharRNNDataset(torch.utils.data.Dataset):
  def __init__(self, text_encoded_windows, train=True):
    self.text = text_encoded_windows
    self.train = train

  def __len__(self):
    return len(self.text)

  def __getitem__(self, ix):
    if self.train:
      return torch.tensor(self.text[ix][:-1]), torch.tensor(self.text[ix][-1])
    return torch.tensor(self.text[ix])

In [31]:
train_text_encoded_windows = windows(train)
test_text_encoded_windows = windows(test)

dataset = {
    'train': CharRNNDataset(train_text_encoded_windows),
    'val': CharRNNDataset(test_text_encoded_windows)
}

dataloader = {
    'train': torch.utils.data.DataLoader(dataset['train'], batch_size=64, shuffle=True, pin_memory=True),
    'val': torch.utils.data.DataLoader(dataset['val'], batch_size=50, shuffle=False, pin_memory=True),
}

len(dataset['train']), len(dataset['val'])

(13304, 3251)

In [32]:
input, output = dataset['train'][1]
tokenizer.seq_to_text(input)

'aron\tRon RonnE Erin\tRonald\nAbel\tAbe Ab Eb EbbE\nAbiel\tBiel Ab\nAbigail\tAb AbbE Gail NabbE\tTabitha\nAbij'

In [33]:
tokenizer.seq_to_text([output])

'a'

In [56]:
class CharRNN(torch.nn.Module):
  def __init__(self, input_size, embedding_size=100, hidden_size=400, num_layers=2, dropout=0.2):
    super().__init__()
    self.encoder = torch.nn.Embedding(input_size, embedding_size)
    self.rnn = torch.nn.LSTM(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, batch_first=True)
    self.fc = torch.nn.Linear(hidden_size, input_size)

  def forward(self, x):
    x = self.encoder(x)
    x, h = self.rnn(x)         
    y = self.fc(x[:,-1,:])
    return y

In [57]:
model = CharRNN(input_size=tokenizer.n_characters)
outputs = model(torch.randint(0, tokenizer.n_characters, (64, 50)))
outputs.shape

torch.Size([64, 100])

In [58]:
from tqdm import tqdm
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

def fit(model, dataloader, epochs=10):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss = []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            bar.set_description(f"loss {np.mean(train_loss):.5f}")
        bar = tqdm(dataloader['val'])
        val_loss = []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                bar.set_description(f"val_loss {np.mean(val_loss):.5f}")
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f}")

def predict(model, X):
    model.eval() 
    with torch.no_grad():
        X = torch.tensor(X).to(device)
        pred = model(X.unsqueeze(0))
        return pred

In [59]:
model = CharRNN(input_size=tokenizer.n_characters)
fit(model, dataloader, epochs=20)

val_loss 2.76481: 100%|██████████| 66/66 [00:00<00:00, 76.32it/s]


Epoch 1/20 loss 2.90289 val_loss 2.76481


val_loss 2.57677: 100%|██████████| 66/66 [00:00<00:00, 77.05it/s]


Epoch 2/20 loss 2.31379 val_loss 2.57677


val_loss 2.52870: 100%|██████████| 66/66 [00:00<00:00, 74.42it/s]


...

Epoch 18/20 loss 0.16269 val_loss 3.58990


val_loss 3.71683: 100%|██████████| 66/66 [00:00<00:00, 75.49it/s]


Epoch 19/20 loss 0.13581 val_loss 3.71683


val_loss 3.94583: 100%|██████████| 66/66 [00:00<00:00, 76.58it/s]

Epoch 20/20 loss 0.11558 val_loss 3.94583


In [60]:
X_new = "Arthur"
X_new_encoded = tokenizer.text_to_seq(X_new)
y_pred = predict(model, X_new_encoded)
y_pred = torch.argmax(y_pred, axis=1)[0].item()
tokenizer.seq_to_text([y_pred])

'\n'

In [61]:
for i in range(4):
  X_new_encoded = tokenizer.text_to_seq(X_new[-100:])
  y_pred = predict(model, X_new_encoded)
  y_pred = torch.argmax(y_pred, axis=1)[0].item()
  X_new += tokenizer.seq_to_text([y_pred])

X_new

'Arthur\nArt'

In [62]:
temp=1
for i in range(100):
  X_new_encoded = tokenizer.text_to_seq(X_new[-100:])
  y_pred = predict(model, X_new_encoded)
  y_pred = y_pred.view(-1).div(temp).exp()
  top_i = torch.multinomial(y_pred, 1)[0]
  predicted_char = tokenizer.all_characters[top_i]
  X_new += predicted_char

print(X_new)

Arthur
Arthene	ArnE HerrE Dena
Ardolia	AllE BaldE
Arminda	BindE ArcE
Arminda	MindE
Arminda	MindE MindE
Arminda
